In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

def get_num_correct(preds, labels):
      return preds.argmax(dim=1).eq(labels).sum().item()
    

class Network(nn.Module):
      def __init__(self):
            super().__init__()
            self.conv1=nn.Conv2d(in_channels=1, out_channels=12, kernel_size=5)#28-4=24-1=23
            self.conv2=nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5)#23-4=19-1=18
            self.conv3=nn.Conv2d(in_channels=24, out_channels=48, kernel_size=5)#18-4=14--->7

            self.fc1=nn.Linear(in_features=48*7*7, out_features=240)
            self.fc2=nn.Linear(in_features=240, out_features=120)
            self.fc3=nn.Linear(in_features=120, out_features=60)
            self.out=nn.Linear(in_features=60, out_features=10)

      def forward(self, t):

            t=F.relu(self.conv1(t))
            t=F.max_pool2d(t, kernel_size=2, stride=1)

            t=F.relu(self.conv2(t))
            t=F.max_pool2d(t, kernel_size=2, stride=1)
            
            t=F.relu(self.conv3(t))
            t=F.max_pool2d(t, kernel_size=2, stride=2)

            t=t.reshape(-1,48*7*7)
            t=F.relu(self.fc1(t))

            t=F.relu(self.fc2(t))
            
            t=F.relu(self.fc3(t))

            t=self.out(t)

            return t
        
        
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
                                   transforms.ToTensor()
    ])
)

In [53]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(5):

    total_loss = 0
    total_correct = 0


    for batch in train_loader:
          images, labels = batch

          preds = network(images)
          loss = F.cross_entropy(preds, labels)

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          total_loss += loss.item()
          total_correct += get_num_correct(preds, labels)

          
    print("epoch:", epoch, "total correct:", total_correct, "loss:", total_loss)

epoch: 0 total correct: 43750 loss: 423.6853380203247
epoch: 1 total correct: 50228 loss: 265.65363985300064
epoch: 2 total correct: 50934 loss: 246.31617929041386
epoch: 3 total correct: 51289 loss: 238.41388738155365
epoch: 4 total correct: 51416 loss: 234.15853591263294


In [49]:
total_correct/60000

0.86345